In [35]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import re
import unidecode as ud
import networkx as nx

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

# tutulo
## subtutulo

t-hoarder lo utilizamos para extraer los datos en un archivo txt. Los datos de felizmartes.txt se han limpiado utilizando el software que Juani mandó para limpiarlos. 

In [36]:
#abrimos el archivo con pandas.
ht = pd.read_csv('felizmartes_clean.txt', sep="\t", on_bad_lines="skip")
#ordenamos los datos por fecha
ht = ht.sort_values(by='date')
#el numero de datos que tenemos (len(ht)) es el número de tweets que tenemos.

ht.to_csv('felizmartes_clean.csv', index=False, header=True)
print('Hemos recolectado',len(ht),'tweets')
ht.head(2)

Hemos recolectado 12647 tweets


,id tweet,date,author,text,app,id user,followers,following,stauses,location,urls,geolocation,name,description,url_media,type media,quoted,relation,replied_id,user replied,retweeted_id,user retweeted,quoted_id,user quoted,first HT,lang,created_at,verified,avatar,link
12646,1567097639883407360,2022-09-06 10:29:27,@zoelaunik,"RT @chelorueda: “El AMOR, más que un te quiero, es un te cuido…” #Buenosdias☕️☕️ #Felizmartes #Seanfelices ♥️🤍 https://t.co/wl8esfmMdg",Twitter for Android,165663614,71,339,5210,recoleta,None,None,زوي,Sin pelos en la lengua!!! Twitter Original & Official® el 24/8 es mi cumpleaños,http://pbs.twimg.com/media/Fb872CiWIAMPh3k.jpg,photo,None,RT,None,None,1567037000045174785,@chelorueda,None,None,Buenosdias,es,2010-07-12 05:58:50,False,https://pbs.twimg.com/profile_images/863371856069447680/PV8A6_r1_normal.jpg,https://twitter.com/zoelaunik/status/1567097639883407360
12645,1567097677770260480,2022-09-06 10:29:36,@fl44444,RT @considerame: TTs [12:20] 🇪🇸 1⃣ #APOYOROCIO6S 2⃣ #FelizMartes 3⃣ #KrashxErik 4⃣ #MartianMTMAD14 5⃣ #ViolaComeIlMareVenezia79 🔝,Twitter for Android,1523320447865081856,95,43,8487,NaN,None,None,flo44,Sencill@ y natural😀,None,None,None,RT,None,None,1567095267077378051,@considerame,None,None,APOYOROCIO6S,und,2022-05-08 15:15:11,False,https://pbs.twimg.com/profile_images/1524793768154628098/v-XPpNWP_normal.jpg,https://twitter.com/fl44444/status/1567097677770260480


In [37]:
#Abrimos el archivo (sacado de gephi?) con los datos de modularidad.
ht_modularity = pd.read_csv('felizmartes_modularidad.csv')
#cogemos las columnas Label y modularity_class
ht_modularity = ht_modularity[['Label', 'modularity_class']]
#renumbramos la columna label a author para tenerlo como antes
ht_modularity.columns=['author', 'modularity_class']
#ponemos las palabras de la columna author en minuscula para estandarizar.
ht_modularity['author'] = ht_modularity['author'].str.lower()
#lo mismo pero en el data set de la columna anterior
ht['author'] = ht['author'].str.lower()
#lo mismo pero con la columna text (el tweet)
ht['text'] = ht['text'].str.lower()
#también ponemos een minuscula la columna user retweeted
ht['user retweeted'] = ht['user retweeted'].str.lower()
#aquí añadimos la columna modularity_class del dataframe que hemos creado al
#data frame de la celda anterior usando la columna author como leyenda (a cada 
# user name le añadimos su correspondiente modularity_class)
ht = ht.merge(ht_modularity, how="left", on="author")

ht.head(2)

,id tweet,date,author,text,app,id user,followers,following,stauses,location,urls,geolocation,name,description,url_media,type media,quoted,relation,replied_id,user replied,retweeted_id,user retweeted,quoted_id,user quoted,first HT,lang,created_at,verified,avatar,link,modularity_class
0,1567097639883407360,2022-09-06 10:29:27,@zoelaunik,"rt @chelorueda: “el amor, más que un te quiero, es un te cuido…” #buenosdias☕️☕️ #felizmartes #seanfelices ♥️🤍 https://t.co/wl8esfmmdg",Twitter for Android,165663614,71,339,5210,recoleta,None,None,زوي,Sin pelos en la lengua!!! Twitter Original & Official® el 24/8 es mi cumpleaños,http://pbs.twimg.com/media/Fb872CiWIAMPh3k.jpg,photo,None,RT,None,None,1567037000045174785,@chelorueda,None,None,Buenosdias,es,2010-07-12 05:58:50,False,https://pbs.twimg.com/profile_images/863371856069447680/PV8A6_r1_normal.jpg,https://twitter.com/zoelaunik/status/1567097639883407360,24.0
1,1567097677770260480,2022-09-06 10:29:36,@fl44444,rt @considerame: tts [12:20] 🇪🇸 1⃣ #apoyorocio6s 2⃣ #felizmartes 3⃣ #krashxerik 4⃣ #martianmtmad14 5⃣ #violacomeilmarevenezia79 🔝,Twitter for Android,1523320447865081856,95,43,8487,NaN,None,None,flo44,Sencill@ y natural😀,None,None,None,RT,None,None,1567095267077378051,@considerame,None,None,APOYOROCIO6S,und,2022-05-08 15:15:11,False,https://pbs.twimg.com/profile_images/1524793768154628098/v-XPpNWP_normal.jpg,https://twitter.com/fl44444/status/1567097677770260480,0.0


In [38]:
#Cogemos la fecha de la columna date, esta viene con la fecha y la hora con minutos y segundos
#lo dividimos en todo lo que viene antes de ':' usando split (sería este el elemento 0 de la lista)
# y el resto detrás de ':', nos quedamos solo con el elemento 0
ht['date_calc'] = ht['date'].apply(lambda x: x.split(":")[0])

print('Mostramos con lo que nos hemos quedado:')
print(ht['date_calc'].head(2))

# añadimos después de la hora que es con lo que nos hemos quedado un :00 para la etiqueta.
ht['date_calc'] = ht['date_calc'].apply(lambda x: x + ":00")

print('Mostramos lo que queda:')
print(ht['date_calc'].head(2))

Mostramos con lo que nos hemos quedado:
0    2022-09-06 10
1    2022-09-06 10
Name: date_calc, dtype: object
Mostramos lo que queda:
0    2022-09-06 10:00
1    2022-09-06 10:00
Name: date_calc, dtype: object


In [39]:
#con esta celda hacemos la distribución por horas
tweets_hour = pd.pivot_table(ht,index= "date_calc",columns= "modularity_class",aggfunc= ["count"],values=['id tweet'])
tweets_hour = tweets_hour.fillna(0)
tweets_hour.to_csv('distribucion_por_horas.csv')

tweets_hour = pd.read_csv('distribucion_por_horas.csv')
tweets_hour.columns = tweets_hour.iloc[1]
tweets_hour = tweets_hour[3:]
tweets_hour.to_csv('distribucion_por_horas.csv')

tweets_hour = pd.read_csv('distribucion_por_horas.csv')
tweets_hour = tweets_hour.drop(['Unnamed: 0'], axis=1)
tweets_hour.rename(columns={'modularity_class':'hour'}, inplace=True)
tweets_hour.head()

,hour,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0
0,2022-09-06 10:00,106.0,16.0,57.0,26.0,2.0,1.0,4.0,4.0,0.0,41.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,4.0,4.0,17.0,9.0,3.0,24.0,0.0,20.0,1.0,1.0,23.0,0.0,3.0,0.0,1.0,0.0,3.0,1.0,1.0
1,2022-09-06 11:00,327.0,132.0,55.0,44.0,8.0,8.0,8.0,0.0,0.0,38.0,3.0,3.0,1.0,2.0,1.0,46.0,0.0,8.0,13.0,27.0,1.0,20.0,25.0,28.0,58.0,0.0,39.0,65.0,2.0,10.0,2.0,2.0,0.0,1.0,0.0,11.0
2,2022-09-06 12:00,547.0,112.0,85.0,40.0,25.0,16.0,18.0,2.0,0.0,19.0,0.0,8.0,2.0,20.0,37.0,45.0,0.0,5.0,6.0,27.0,0.0,6.0,31.0,17.0,62.0,15.0,57.0,13.0,6.0,13.0,3.0,20.0,13.0,2.0,0.0,7.0
3,2022-09-06 13:00,498.0,58.0,96.0,83.0,25.0,30.0,44.0,58.0,0.0,4.0,0.0,0.0,0.0,23.0,7.0,44.0,0.0,5.0,5.0,14.0,1.0,11.0,26.0,57.0,47.0,8.0,55.0,7.0,3.0,5.0,1.0,6.0,0.0,1.0,0.0,4.0
4,2022-09-06 14:00,310.0,19.0,93.0,43.0,15.0,24.0,30.0,32.0,1.0,41.0,2.0,1.0,11.0,3.0,9.0,46.0,4.0,1.0,1.0,4.0,21.0,7.0,29.0,27.0,65.0,18.0,25.0,12.0,1.0,2.0,6.0,2.0,1.0,4.0,0.0,9.0


In [49]:
#Ahora vamos a separar los tweets en palabras
#para ello vamos a quedarnos solo con el id del tweet, el author y el texto.

#cogemos solo una selección de tweets para la prueba
ht_s=ht.sample(10000,random_state=987)

data=pd.DataFrame()
data['id tweet']=ht_s['id tweet']
data['author']=ht_s['author']
data['text']=ht_s['text']
data.head()

,id tweet,author,text
2029,1567124611871178753,@eafu2011,rt @rancaguab: #felizmartes hoy recordamos que si las calles no tienen #ciclovias el ciclista puede usarla y los vehículos motorizados tienen que pasar a 1.5 m de distancia. #todoenbicicleta #pedalpower https://t.co/pqcrkzb5bo
7454,1567201810909110275,@sorelladomenica,"rt @arteviajero_com: ¿alguna vez has visto una iglesia sobre un volcán? en la localidad francesa de aiguilhe (haute-loire, auvergne) es posible contemplar una de las estampas más espectaculares del arte románico de francia #felizmartes #travel https://t.co/shh6dggyxl"
3526,1567139218958213120,@oficinaato,rt @bellayurimar1: #felizmartes ‼️𝗡𝘂𝗲𝘃𝗼 𝘀𝗶𝘀𝘁𝗲𝗺𝗮 𝗱𝗲 𝗽𝗮𝗴𝗼‼️ 🔲la plataforma patria reactiva puntos de venta 𝘃𝗲𝗣𝗢𝗦 para el uso del petro en varios comercios 𝗱𝗲𝗹 𝗽𝗮í𝘀 🇻🇪 #6sep https://t.co/myogogajz7
7043,1567193650848727042,@anamari90358533,rt @finanzasenfa: si hay buenas!! escucha en este martes de #championsleague el resumen con @marianoosorio1 sobre el empleo del #imss y la confianza del consumidor! https://t.co/cwxc8tah26 #creditos #bienparabien @bienparabien #felizmartesatodos #felizmartes
2338,1567128221044318211,@mmariophoto,"rt @teidagua: 🔵 ""azul, a su lado"", calle obispo rey redondo #lalaguna #escenasdedepatrimonio 🤗¡muy #felizmartes! https://t.co/5ri78xyhes"


In [50]:
#Para separar por palabras:
# Crear una lista vacía para almacenar los datos limpios
datos_limpios = []

# Iterar a través de cada fila del conjunto de datos original
for i, row in data.iterrows():
    # Dividir el tweet en palabras
    palabras = row['text'].split()
    
    # Iterar a través de cada palabra y agregar una nueva fila al conjunto de datos limpios
    for palabra in palabras:
        datos_limpios.append({'id tweet': row['id tweet'], 'author': row['author'], 'palabra': palabra})
        
# Convertir la lista de diccionarios en un DataFrame de pandas
datos_clean = pd.DataFrame(datos_limpios)

# Exportar el archivo como CSV
#false es para que no guarde los índice de las distintas filas en el csv 
datos_clean.to_csv('felizmartes_words.csv', index=False)
print(len(datos_clean))
datos_clean.head()

260115


,id tweet,author,palabra
0,1567124611871178753,@eafu2011,rt
1,1567124611871178753,@eafu2011,@rancaguab:
2,1567124611871178753,@eafu2011,#felizmartes
3,1567124611871178753,@eafu2011,hoy
4,1567124611871178753,@eafu2011,recordamos


Se va a quitar ruido de la red eliminando las palabras que no nos den información (preposiciones, nexos, el hashtag principal). Ordenamos por grado los nodos de las palabras y eliminamos los de mayor grado (los mencionados) a no ser que veamos que existen comunidades dividas por esa palabra.  

In [51]:
#Ahora vamos a sacar las aristas de estos datos con el código que teniamos anteriormente

all_edges = [] #guardamos las aristas con los pesos
all_edges_aux = [] #guardamos las aristas sin los pesos


for i, row in datos_clean.iterrows():
    edge=(row['author'],row['palabra'])
    # Condicional para validar si ya existe un enlace y agregarlo como peso
    if (edge[0], edge[1]) in all_edges_aux:
        edge_index = all_edges_aux.index((edge[0], edge[1]))
        # Se suma 1 al peso anterioir que tenía el enlace
        all_edges[edge_index] = (edge[0], edge[1], all_edges[edge_index][2] + 1)
    else:
        # Se agrega el nuevo enlace al auxiliar y al original. En el arreglo original de enlaces se inicia
        # con un peso de 1
        all_edges.append((edge[0], edge[1],1))
        all_edges_aux.append((edge[0], edge[1]))

# Total de enlaces de la red
total_count_edges = len(all_edges)

# Arreglos para almacenar nodos
all_nodes = set()
users_nodes = set()
words_nodes = set()

# Proceso para obtener los nodos de la red, x=usuario, y=palabra, w=peso
for x,y,w in all_edges:
    users_nodes.add(x)
    words_nodes.add(y)
    all_nodes.add(x)
    all_nodes.add(y)

print("El tamaño de la red está definido por los siguientes valores:")
print("Total de enlaces:", len(all_edges))
print("Total de nodos:", len(all_nodes))
print("Total de nodos usuario:", len(users_nodes))
print("Total de nodos de palabras:", len(words_nodes))



In [46]:
B = nx.Graph() #Inicializamos el grafo como B (utilizando la librería networkx)

words = set()   #inicializamos las palabras.
for user in users_nodes:    #iteramos para todos los usuarios que guardamos anteriormente en users_nodes
    B.add_node(user, bipartite=0)   #creamos los nodos de usuario del grafo

for word in words_nodes:
    B.add_node(word, bipartite=1)   #creamos los nodos de palabra del grafo


for edge in all_edges: #iteramos para cada arista 
    B.add_edge(edge[0], edge[1], weight=edge[2])
    #estamos añadiendo al grafo las arista: usuario, palabra, peso

In [47]:
# Obtenemos la proyección de palabras
word_projection = nx.bipartite.weighted_projected_graph(B, words_nodes)

nx.write_gexf(word_projection, "proyeccion_palabras_data.gexf")

In [48]:
# Obtenemos la proyección de usuarios
users_projection = nx.bipartite.weighted_projected_graph(B, users_nodes)

nx.write_gexf(users_projection, "proyeccion_usuarios_data.gexf")